In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
# This file is a simple implementation of the 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
model = Sequential()
data_file = pd.read_csv(r"/storage/eurusd_train_normed.csv")
eurusd_test = pd.read_csv(r"/storage/eurusd_test_normed.csv")
data_file = eurusd_train.replace(np.nan, 0)
eurusd_test = eurusd_test.replace(np.nan, 0)

########################### Set Model Paramaters #############################
# this looks back over a set period as the memory for the LSTM
look_back = 60
features_to_use = ["spot_v_HF", "spot_v_MF", "spot_v_LF", "HF_ema_diff",
                   "MF_ema_diff","LF_ema_diff", "LDN", "NY", "Asia", "target"]
data_size = 475
# the length of our prediction
trade_horizon = 24
# create data_set
train , test= create_train_test_file(data_file, data_size, test_split, model_features):
train_sample = train.values
test_sample = test.values
 # Parse the values into the LSTM format
train_data , train_target = create_dataset(train_sample,False, look_back)
test_data, test_target, target_dates = create_dataset(test_sample, True, look_back)

# reshape seems to add another list around every observation
train_data = train_data.reshape(train_data.shape[0], look_back, train_data.shape[2])
train_target = train_target.reshape(train_target.shape[0], 1)
test_data = test_data.reshape(test_data.shape[0], look_back, test_data.shape[2])
test_target = test_target.reshape(test_target.shape[0], 1)
# Build up the model
BATCH_SIZE = 300
no_features = train_data.shape[2]
model = Sequential()
model.add(LSTM(4,batch_input_shape = (None,look_back,no_features), return_sequences = True))
model.add(LSTM(1, return_sequences = False, activation="softmax"))
#model.add(Dense(2, activation= "softmax"))
model.compile(loss = "mean_absolute_error", optimizer="adam", metrics = ['accuracy'])
model.summary()
EPOCH = 1000
# train the model
# verbose = 1 gives the output of the training.
lstm_engine = model.fit(train_data,train_target,epochs = EPOCH,validation_data=(test_data,test_target), verbose=0)
# run training on the test data
results = model.predict(test_data)
# The % threshold needed to trigger a signal either way
predicted = [signal(i, 0.55) for i in results]
acc_score = get_accuracy(predicted, test_target)
predictions = pd.DataFrame({"Date" : target_dates,"Predictions": predicted})
test_original = pd.merge(test,predictions,how="left", on="Date").fillna(0)
# calculate the returns of the signal
test_original["scaled_signal"] = test_original['Predictions'].shift(2).rolling(trade_horizon).sum()/trade_horizon
# no shift needed as we have already done that in previous step
test_original['strat_returns'] = test_original['logret']*test_original['scaled_signal']
test_original['strat_returns_sum'] = test_original['strat_returns'].cumsum()
strat_return = test_original['strat_returns'].sum()
information_ratio = (test_original['strat_returns'].mean()*260)/(test_original['strat_returns'].sd()*np.sqrt(260))


In [ ]:
# mergeing signals to the df
predictions = pd.DataFrame({"Date" : target_dates,"Predictions": predicted})
test_original = pd.merge(test_original,predictions,how="left", on="Date").fillna(0)
test_original.columns

In [ ]:
trade_horizon = 24
test_original["scaled_signal"] = test_original['Predictions'].shift(2).rolling(trade_horizon).sum()/trade_horizon
# no shift needed as we have already done that inpervious step
test_original['strat_returns'] = test_original['logret']*test_original['scaled_signal']
test_original['strat_returns_sum'] = test_original['strat_returns'].cumsum()

In [ ]:
ax1 = plt.plot()
s1 = test_original['CCY']
plt.plot(s1,'b')
plt.ylabel('Ret',color='b')
ax2 = plt.gca().twinx()
s2 = test_original['strat_returns_sum'].loc[test_original['scaled_signal'] <0]
ax2.plot(s2, 'r*')
plt.ylabel('sin', color='r')
s3 = test_original['strat_returns_sum'].loc[test_original['scaled_signal'] >0]
ax2.plot(s3, 'g*')
plt.show()
test_original.to_csv(r"/storage/signal_prediction_test.csv",index = False)